In [1]:
%matplotlib notebook
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
import seaborn as sns
import colorlover as cl

py.offline.init_notebook_mode(connected=True)

In [2]:
fname0 = '../../../../../Analysis/mom/FAMOS_GS/Analysis/data/ITU-MOM/ITU-MOM_GSexps_1980_2009.nc'
ds0 = xr.open_mfdataset(fname0)

fname1 = '../../../../../Analysis/mom/FAMOS_GS/Analysis/data/MITB500/MIT_MITB500_1980_2009.nc'
ds1 = xr.open_mfdataset(fname1)

fname2 = '../../../../../Analysis/mom/FAMOS_GS/Analysis/data/Brest/Brest_GSexps_1980_2009.nc'
ds2 = xr.open_mfdataset(fname2)

fname3 = '../../../../../Analysis/mom/FAMOS_GS/Analysis/data/NorESM/NorESM_GS_experiments_1980_2009.nc'
ds3 = xr.open_mfdataset(fname3)

fname4 = '../../../../../Analysis/mom/FAMOS_GS/Analysis/data/FESOM/FESOM_GS_exps_1980_2009.nc'
ds4 = xr.open_mfdataset(fname4)

In [3]:
dates0 = pd.date_range('1980-01', freq='M', periods=12*29)
ds0['time'] = dates0
ds1['time'] = dates0
ds2['time'] = dates0
ds3['time'] = dates0
ds4['time'] = dates0

In [10]:
""" Plot annual volume vs heat anomaly"""
modelnames = ['ITU' , 'MITB500', 'Brest', 'NorESM', 'FESOM']
#colors = ['blue', 'darkorange', 'red', 'green', 'cyan']
colors = cl.scales['5']['qual']['Dark2']; 

def plot_bso_inflow_vol_heat(models):
    
    datap = []
    datan = []
    layout = go.Layout(
    title='<b>Annual inflow volume vs heat transport anomaly @BSO</b>',
        yaxis=dict(
            title='Heat transport [TW]',
            #range=[-14, 14]
        ),
        xaxis=dict(
            title='Volume transport [Sv]'
        )
    )  
    # names of the control, gsp and gsn variables [total]
    ctln = ['ctlbarents_volumeinflow', 'barvt_c[1]', 'BSOrefvolume_in', 'inflow_vt_NorESM_BSO_ctrl', 'ctlvoltransp_BSO_in']
    gspn = ['gspbarents_volumeinflow', 'barvt_gsp[1]', 'BSOgspvolume_in', 'inflow_vt_NorESM_BSO_gsp', 'gspvoltransp_BSO_in']
    gsnn = ['gsnbarents_volumeinflow', 'barvt_gsm[1]', 'BSOgsnvolume_in', 'inflow_vt_NorESM_BSO_gsn', 'gsnvoltransp_BSO_in']

    ctlhn = ['ctlbarents_heatinflow', 'barht_c[1]', 'BSOrefheat_in', 'inflow_ht_NorESM_BSO_ctrl', 'ctlheattransp_BSO_in']
    gsphn = ['gspbarents_heatinflow', 'barht_gsp[1]', 'BSOgspheat_in', 'inflow_ht_NorESM_BSO_gsp', 'gspheattransp_BSO_in']
    gsnhn = ['gsnbarents_heatinflow', 'barht_gsm[1]', 'BSOgsnheat_in', 'inflow_ht_NorESM_BSO_gsn', 'gsnheattransp_BSO_in']

    coeffs = [1, 1e-6, 1, 1, 1]
    for ind,mdl in enumerate(models):
        dnm = 'ds'+np.str(ind)+'.'+ctln[ind]+'.groupby(\'time.year\').mean().year'
        time = np.copy(eval(dnm))
        dnm = 'ds'+np.str(ind)+'.'+ctln[ind]+'.groupby(\'time.year\').mean()'
        dnm2 = 'ds'+np.str(ind)+'.'+gspn[ind]+'.groupby(\'time.year\').mean()'
        dnm3 = 'ds'+np.str(ind)+'.'+gsnn[ind]+'.groupby(\'time.year\').mean()'
        val1 = (np.copy(eval(dnm2)-eval(dnm))*coeffs[ind]) #/(np.copy(eval(dnm))*coeffs[ind])
        val2 = (np.copy(eval(dnm3)-eval(dnm))*coeffs[ind]) #/(np.copy(eval(dnm))*coeffs[ind])
        dnm = 'ds'+np.str(ind)+'.'+ctlhn[ind]+'.groupby(\'time.year\').mean()'
        dnm2 = 'ds'+np.str(ind)+'.'+gsphn[ind]+'.groupby(\'time.year\').mean()'
        dnm3 = 'ds'+np.str(ind)+'.'+gsnhn[ind]+'.groupby(\'time.year\').mean()'
        val3 = np.copy(eval(dnm2)-eval(dnm))
        val4 = np.copy(eval(dnm3)-eval(dnm))
        trace1 = go.Scatter(            
            x = val1,
            y = val3,
            mode = 'lines',
            name = mdl+'-GSP',
            line = dict(
                width = 2.5,
                color = colors[ind],
                dash = 'dot',
                shape = 'smooth'
            )
        )
        datap.append(trace1)
        trace2 = go.Scatter(            
            x = val2,
            y = val4,
            mode = 'lines',
            name = mdl+'-GSM',
            line = dict(
                width = 2.5,
                color = colors[ind],
                dash = 'dashdot',
                shape = 'smooth'
            )
        )
        datan.append(trace2)


   
    data = datap + datan
    
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)
    

def plot_bso_total_vol_heat(models):
    
    datap = []
    datan = []
    layout = go.Layout(
    title='<b>Annual total volume vs heat transport anomaly @BSO</b>',
        yaxis=dict(
            title='Heat transport [TW]',
            #range=[-14, 14]
        ),
        xaxis=dict(
            title='Volume transport [Sv]'
        )
    )  
    # names of the control, gsp and gsn variables [total]
    ctln = ['ctlbarents_volumetotal', 'barvt_c[0]', 'BSOrefvolume', 'vt_NorESM_BSO_ctrl', 'ctlvoltransp_BSO']
    gspn = ['gspbarents_volumetotal', 'barvt_gsp[0]', 'BSOgspvolume', 'vt_NorESM_BSO_gsp', 'gspvoltransp_BSO']
    gsnn = ['gsnbarents_volumetotal', 'barvt_gsm[0]', 'BSOgsnvolume', 'vt_NorESM_BSO_gsn', 'gsnvoltransp_BSO']
    
    ctlhn = ['ctlbarents_heattotal', 'barht_c[0]', 'BSOrefheat', 'ht_NorESM_BSO_ctrl', 'ctlheattransp_BSO']
    gsphn = ['gspbarents_heattotal', 'barht_gsp[0]', 'BSOgspheat', 'ht_NorESM_BSO_gsp', 'gspheattransp_BSO']
    gsnhn = ['gsnbarents_heattotal', 'barht_gsm[0]', 'BSOgsnheat', 'ht_NorESM_BSO_gsn', 'gsnheattransp_BSO']

    coeffs = [1, 1e-6, 1, 1, 1]
    for ind,mdl in enumerate(models):
        dnm = 'ds'+np.str(ind)+'.'+ctln[ind]+'.groupby(\'time.year\').mean().year'
        time = np.copy(eval(dnm))
        dnm = 'ds'+np.str(ind)+'.'+ctln[ind]+'.groupby(\'time.year\').mean()'
        dnm2 = 'ds'+np.str(ind)+'.'+gspn[ind]+'.groupby(\'time.year\').mean()'
        dnm3 = 'ds'+np.str(ind)+'.'+gsnn[ind]+'.groupby(\'time.year\').mean()'
        val1 = (np.copy(eval(dnm2)-eval(dnm))*coeffs[ind]) #/(np.copy(eval(dnm))*coeffs[ind])
        val2 = (np.copy(eval(dnm3)-eval(dnm))*coeffs[ind]) #/(np.copy(eval(dnm))*coeffs[ind])
        dnm = 'ds'+np.str(ind)+'.'+ctlhn[ind]+'.groupby(\'time.year\').mean()'
        dnm2 = 'ds'+np.str(ind)+'.'+gsphn[ind]+'.groupby(\'time.year\').mean()'
        dnm3 = 'ds'+np.str(ind)+'.'+gsnhn[ind]+'.groupby(\'time.year\').mean()'
        val3 = np.copy(eval(dnm2)-eval(dnm))
        val4 = np.copy(eval(dnm3)-eval(dnm))
        trace1 = go.Scatter(            
            x = val1,
            y = val3,
            mode = 'lines',
            name = mdl+'-GSP',
            line = dict(
                width = 2.5,
                color = colors[ind],
                dash = 'dot',
                shape = 'smooth'
            )
        )
        datap.append(trace1)
        trace2 = go.Scatter(            
            x = val2,
            y = val4,
            mode = 'lines',
            name = mdl+'-GSM',
            line = dict(
                width = 2.5,
                color = colors[ind],
                dash = 'dashdot',
                shape = 'smooth'
            )
        )
        datan.append(trace2)


   
    data = datap + datan
    
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)    


plot_bso_inflow_vol_heat(modelnames)
plot_bso_total_vol_heat(modelnames)


#models = widgets.SelectMultiple(options=list(modelnames), value=modelnames, description='Models')    
#widgets.interactive(update_plot,models=models)